In [1]:
# !pip install keras-layer-normalization
# !pip install keras_multi_head
# !pip install keras_position_wise_feed_forward
# !pip install keras_embed_sim
# !pip install keras-self-attention
# !pip install keras_ordered_neurons
# !pip install keras-pos-embd

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.layers import Embedding, Masking, Concatenate, GRU, Dense, Reshape
from other_model2 import prosenet_model, bilstm, onlstm, transformer, deepmoji

from sklearn.metrics import accuracy_score, roc_auc_score
from evaluate import calculate_BS, f1, show_eval_result_2class

from typing import Tuple, Dict, Any

In [2]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [3]:
df = pd.read_csv('new_test.csv')
df = pd.get_dummies(df)

In [4]:
# transformer 需要的 redundant col
df.insert(0, column='redundant', value=0.0)

In [5]:
def get_rally_result(df):
    result_label = []
    ignor_game_rally_index = []
    team_col = [c for c in df.columns if 'Team' in c]
    for _, df_rally in df.groupby(['Game', 'Rally']):
        start_from = df.iloc[df_rally.index[0]][team_col].tolist()

        result = df_rally[df_rally['Score'] == 1]
        if(len(result) != 0):
            if(result.iloc[0][team_col].tolist() == start_from):
                result_label.append([0, 1])
            else:
                result_label.append([1, 0])
        else:
            result = df_rally[df_rally['Errors'] == 1]
            if(len(result) == 0):
                ignor_game_rally_index.append((df_rally.iloc[0]['Game'], df_rally.iloc[0]['Rally']))
            elif(result.iloc[0][team_col].tolist() != start_from):
                result_label.append([0, 1])
            else:
                result_label.append([1, 0])
    return tf.constant(result_label, dtype=float), ignor_game_rally_index

In [6]:
df.columns

Index(['redundant', 'Game', 'Rally', 'Errors', 'Nothing', 'Score',
       'Player_BRA_1', 'Player_BRA_11', 'Player_BRA_12', 'Player_BRA_14',
       ...
       'Action_At_0', 'Action_B_0', 'Action_D_0', 'Action_D_1', 'Action_FS_0',
       'Action_G_0', 'Action_G_1', 'Action_JS_0', 'Action_R_0', 'Action_R_1'],
      dtype='object', length=175)

In [7]:
def get_padding_data(df, ignor):
    result_col = ['Errors', 'Score', 'Nothing']
    team_col = [c for c in df.columns if 'Team' in c]
    col = [c for c in df.columns if c not in result_col and c != 'Game' and c != 'Rally' and c not in team_col]

    rally_set = []
    for _, df_rally in df.groupby(['Game', 'Rally']):   # each rally in one game
        if((df_rally.iloc[0]['Game'], df_rally.iloc[0]['Rally']) in ignor):
            continue
        curr_team = df.iloc[df_rally.index[0]][team_col].tolist()
        shot_set = []
        atk_sequence = []
        
        for _, shot in df_rally.iterrows():
            if(shot[team_col].tolist() != curr_team):
                shot_set.append(atk_sequence)
                
                curr_team = shot[team_col].tolist()
                atk_sequence = []

            atk_sequence.append(shot[col])
        
        # the last shot
        shot_set.append(atk_sequence)

        # one rally has been finished
        shot_set = pad_sequences(shot_set, maxlen=3, padding='post')

        # one rally has been finished
        rally_set.append(shot_set)

    padded_rally_set = pad_sequences(rally_set, dtype=float, padding='post')
    
    return padded_rally_set


In [8]:
label, ignor = get_rally_result(df)

In [9]:
rally_set = get_padding_data(df, ignor)

In [10]:
# rally數, 最大回合數in one rally, 3, feature數
print(rally_set.shape)
print(label.shape)

(1291, 14, 3, 161)
(1291, 2)


---

In [11]:
rally_set_tensor = tf.convert_to_tensor(rally_set)
rally_result_tensor = tf.convert_to_tensor(label)

In [12]:
def split_data(others_tensor, label_tensor):
    l = label_tensor.shape[0]
    split_persentage = int(l*0.7)

    train_x = others_tensor[:split_persentage]
    train_y = label_tensor[:split_persentage]

    test_x = others_tensor[split_persentage:]
    test_y = label_tensor[split_persentage:]

    return train_x, train_y, test_x, test_y

In [13]:
train_x, train_y, test_x, test_y = split_data(rally_set_tensor, rally_result_tensor)

In [14]:
def eval_model_result(train_x, train_y, test_x, test_y, 
                        model_kwargs: Dict[str, Any] = {'model': None, 'optimizer': None, 'loss': None, 'metrics': None, 'epochs': None, 'shapes': []}):
    
    optimizer = model_kwargs['optimizer']
    loss = model_kwargs['loss']
    metrics = model_kwargs['metrics']
    epochs = model_kwargs['epochs']
    shapes = model_kwargs['shapes']
    callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True, monitor='val_loss')

    if(model_kwargs['model'] == 'prosenet_model'):
        model = prosenet_model(shot_sequence_shape=shapes)
    elif(model_kwargs['model'] == 'deepmoji'):
        model = deepmoji(shot_sequence_shape=shapes)
    elif(model_kwargs['model'] == 'bilstm'):
        model = bilstm(shot_sequence_shape=shapes)
    elif(model_kwargs['model'] == 'onlstm'):
        model = onlstm(shot_sequence_shape=shapes)
    elif(model_kwargs['model'] == 'transformer'):
        model = transformer(shot_sequence_shape=shapes)

    model.summary()
    print('\n' + '='*50 + 'training' + '='*50)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    model.fit(train_x, train_y, epochs=epochs, validation_split=0.1, callbacks=[callbacks])

    print('\n' + '='*50 + 'test result' + '='*50)
    model.evaluate(test_x, test_y)

    print('\n' + '='*50 + 'eval result' + '='*50)
    y_pred = model.predict(test_x)
    argmax_y_pred_reshape = np.argmax(y_pred, axis=1)
    argmax_test_y_reshape = np.argmax(test_y, axis=1)

    acc_score = accuracy_score(argmax_test_y_reshape, argmax_y_pred_reshape)
    f1_score = f1(argmax_test_y_reshape, argmax_y_pred_reshape)
    auc_score = roc_auc_score(test_y, y_pred, multi_class='ovr')
    BS = calculate_BS(test_y, y_pred, 2)
    show_eval_result_2class(acc_score, f1_score, auc_score, BS)

In [35]:
eval_model_result(train_x, train_y, test_x, test_y, 
                  model_kwargs={'model': 'bilstm', 
                                'optimizer': 'adam', 
                                'loss': keras.losses.CategoricalCrossentropy(), 
                                'metrics': ['accuracy'], 
                                'epochs': 30, 
                                'shapes': rally_set.shape[1:]})

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Shots_input (InputLayer)     [(None, 14, 3, 160)]      0         
_________________________________________________________________
Sequence_masking (Masking)   (None, 14, 3, 160)        0         
_________________________________________________________________
cnn_with_mask (CNN_with_mask (None, 14, 1, 160)        76960     
_________________________________________________________________
tf.compat.v1.squeeze (TFOpLa (None, 14, 160)           0         
_________________________________________________________________
Bidirectional_recurrent_laye (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 126,498
Trainable params: 126,498
Non-trainable params: 0
_______________________________________________________

---

In [16]:
# train_y_1D = np.argmax(train_y, axis=1)
# test_y_1D = np.argmax(test_y, axis=1)

In [36]:
eval_model_result(train_x, train_y, test_x, test_y, 
                  model_kwargs={'model': 'deepmoji', 
                                'optimizer': 'adam', 
                                'loss': keras.losses.CategoricalCrossentropy(), 
                                'metrics': ['accuracy'], 
                                'epochs': 30, 
                                'shapes': rally_set.shape[1:]})

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Shots_input (InputLayer)        [(None, 14, 3, 160)] 0                                            
__________________________________________________________________________________________________
Sequence_masking (Masking)      (None, 14, 3, 160)   0           Shots_input[0][0]                
__________________________________________________________________________________________________
cnn_with_mask_1 (CNN_with_mask) (None, 14, 1, 160)   76960       Sequence_masking[0][0]           
__________________________________________________________________________________________________
tf.compat.v1.squeeze_1 (TFOpLam (None, 14, 160)      0           cnn_with_mask_1[0][0]            
____________________________________________________________________________________________

In [37]:
eval_model_result(train_x, train_y, test_x, test_y, 
                  model_kwargs={'model': 'onlstm', 
                                'optimizer': 'adam', 
                                'loss': keras.losses.CategoricalCrossentropy(), 
                                'metrics': ['accuracy'], 
                                'epochs': 30, 
                                'shapes': rally_set.shape[1:]})

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Shots_input (InputLayer)     [(None, 14, 3, 160)]      0         
_________________________________________________________________
Sequence_masking (Masking)   (None, 14, 3, 160)        0         
_________________________________________________________________
cnn_with_mask_2 (CNN_with_ma (None, 14, 1, 160)        76960     
_________________________________________________________________
tf.compat.v1.squeeze_2 (TFOp (None, 14, 160)           0         
_________________________________________________________________
ONLSTM (ONLSTM)              (None, 32)                27792     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 104,818
Trainable params: 104,818
Non-trainable params: 0
_____________________________________________________

In [38]:
eval_model_result(train_x, train_y, test_x, test_y, 
                  model_kwargs={'model': 'transformer', 
                                'optimizer': 'adam', 
                                'loss': keras.losses.CategoricalCrossentropy(), 
                                'metrics': ['accuracy'], 
                                'epochs': 30, 
                                'shapes': rally_set.shape[1:]})

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Shots_input (InputLayer)        [(None, 14, 3, 160)] 0                                            
__________________________________________________________________________________________________
Sequence_masking (Masking)      (None, 14, 3, 160)   0           Shots_input[0][0]                
__________________________________________________________________________________________________
cnn_with_mask_3 (CNN_with_mask) (None, 14, 1, 160)   76960       Sequence_masking[0][0]           
__________________________________________________________________________________________________
tf.compat.v1.squeeze_3 (TFOpLam (None, 14, 160)      0           cnn_with_mask_3[0][0]            
____________________________________________________________________________________________

In [15]:
eval_model_result(train_x, train_y, test_x, test_y, 
                  model_kwargs={'model': 'prosenet_model', 
                                'optimizer': 'adam', 
                                'loss': keras.losses.CategoricalCrossentropy(), 
                                'metrics': ['accuracy'], 
                                'epochs': 30, 
                                'shapes': rally_set.shape[1:]})

Model: "Classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Shots_input (InputLayer)     [(None, 14, 3, 161)]      0         
_________________________________________________________________
Sequence_masking (Masking)   (None, 14, 3, 161)        0         
_________________________________________________________________
cnn_with_mask (CNN_with_mask (None, 14, 1, 161)        77924     
_________________________________________________________________
tf.compat.v1.squeeze (TFOpLa (None, 14, 161)           0         
_________________________________________________________________
prototypes (Prototypes)      (None, 14, 16)            2576      
Total params: 80,500
Trainable params: 80,500
Non-trainable params: 0
_________________________________________________________________

==================================================training==================================================
Epoc

ValueError: in user code:

    c:\Users\YiChen\anaconda3\envs\SportsScience\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    c:\Users\YiChen\anaconda3\envs\SportsScience\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\Users\YiChen\anaconda3\envs\SportsScience\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\Users\YiChen\anaconda3\envs\SportsScience\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\Users\YiChen\anaconda3\envs\SportsScience\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\Users\YiChen\anaconda3\envs\SportsSci